In [17]:
import os
import pymysql
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings('ignore')

## Connecting to Database

In [18]:
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = "cricket123"
database = os.getenv('MYSQL_DATABASE')

In [19]:
conn = pymysql.connect(
    host = host,
    port = port,
    user = "root",
    passwd = password,
    db = "happiness")

## Streamlit App Misc.

### In order to create a dropdown menu with the different countries and regions options, below is how I got a list of countries/regions

In [9]:
# used the year 2015 because it had the most countries among the years
country = pd.read_sql_query("SELECT DISTINCT `Country` FROM `happiness`.`2015` ORDER BY `Country` ASC;", conn)
countries_list = country['Country'].to_list()
print(countries_list)

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivory Coast', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania', 'Luxembourg', 'Macedoni

In [10]:
region = pd.read_sql_query("SELECT DISTINCT `Region` FROM `happiness`.`2015` ORDER BY `Region` ASC;", conn)
regions_list = region['Region'].to_list()
print(regions_list)

['Australia and New Zealand', 'Central and Eastern Europe', 'Eastern Asia', 'Latin America and Caribbean', 'Middle East and Northern Africa', 'North America', 'Southeastern Asia', 'Southern Asia', 'Sub-Saharan Africa', 'Western Europe']


## Question #1

### Finds the rank difference of the selected country against the two different years selected

#### Step 1: Choose the year and country wanted

In [22]:
# YEARS TO CHOOSE FROM: 2015, 2016, 2017, 2018, 2019 (make sure to put into string form)
starting_year = "2015"
ending_year = "2019"

In [23]:
country = "Switzerland"
query1 = "SELECT happiness.`" + starting_year + "`.`country`, happiness.`" + \
          starting_year + "`.`Happiness Rank` - happiness.`" + ending_year + \
          "`.`Happiness Rank` AS 'Happiness Rank Difference' FROM happiness.`" + \
          ending_year + "` INNER JOIN happiness.`" + starting_year + \
          "` ON happiness.`" + ending_year + "`.`Country` = happiness.`" + starting_year + \
          "`.`Country` WHERE happiness.`" + ending_year + "`.`Country` = '" + country + "'"
df1 = pd.read_sql_query(query1, conn)
df1

,country,Happiness Rank Difference
0,Switzerland,-5


## Question #2

### Finds the average rank difference amongst countries in the selected region against the two different years selected

In [24]:
# YEARS TO CHOOSE FROM: 2015, 2016, 2017, 2018, 2019 (make sure to put into string form)
starting_year = "2015"
ending_year = "2018"

In [25]:
region = "North America"
query2 = "SELECT happiness.`" + starting_year + "`.`region`, AVG(happiness.`" + \
          starting_year + "`.`Happiness Rank` - happiness.`" + ending_year + \
          "`.`Happiness Rank`) AS 'Happiness Rank Difference' FROM happiness.`" + \
          ending_year + "` INNER JOIN happiness.`" + starting_year + "` ON happiness.`" + \
          ending_year + "`.`Country` = happiness.`" + starting_year + "`.`Country` WHERE happiness.`" + \
          starting_year + "`.`region` = '" + region + "' GROUP BY `" + starting_year + "`.`region`"
df2 = pd.read_sql_query(query2, conn)
df2

,region,Happiness Rank Difference
0,North America,-2.5


## Question 3

### Out of curiosity, I wanted to see the breakdown of the factors that impacts the Happiness Rank and Score

In [ ]:
starting_year = "2015"
ending_year = "2019"
country = 'Netherlands'

In [63]:
query3 = "SELECT happiness.`" + starting_year + "`.`country`,(happiness.`" + starting_year + \
"`.`Happiness Rank` - happiness.`" + ending_year + "`.`Happiness Rank`) AS 'Happiness Rank Difference', ROUND(happiness.`" \
+ ending_year + "`.`Happiness Score` - happiness.`" + starting_year + \
"`.`Happiness Score`, 4) AS 'Happiness Score Difference', ROUND(happiness.`" + ending_year + \
"`.`Economy` - happiness.`" + starting_year + "`.`Economy`, 4) AS 'Economic Difference', ROUND(happiness.`" + \
ending_year + "`.`Health` - happiness.`" + starting_year + "`.`Health`, 4) AS 'Health Difference', ROUND(happiness.`" + \
ending_year + "`.`Freedom` - happiness.`" + starting_year + "`.`Freedom`, 4) AS 'Freedom Difference', ROUND(happiness.`" + \
ending_year + "`.`Government` - happiness.`" + starting_year + "`.`Government`, 4) AS 'Governmental Difference' FROM happiness.`" + \
ending_year + "`INNER JOIN happiness.`" + starting_year + "`ON happiness.`" + \
ending_year + "`.`Country` = happiness.`" + starting_year + "`.`Country` WHERE `" + starting_year + "`.`Country` = '" + country + "'"

diff = pd.read_sql_query(query3, conn)
diff

,country,Happiness Rank Difference,Happiness Score Difference,Economic Difference,Health Difference,Freedom Difference,Governmental Difference
0,Netherlands,2,0.11,0.0666,0.1062,-0.0588,-0.0201


### Interactive Plot: to visualize difference in economy, health, freedom, and government
#### - click left key for individual categories and hold shift for multiple - 

In [64]:
start = "SELECT `Country`, `Happiness Rank`, `Happiness Score`, `Economy`, `Health`, `Freedom`, `Government` FROM `Happiness`.`" + starting_year + "` WHERE `Country` = '" + country + "'"
end = "SELECT `Country`, `Happiness Rank`, `Happiness Score`, `Economy`, `Health`, `Freedom`, `Government` FROM `Happiness`.`" + ending_year + "` WHERE `Country` = '" + country + "'"

start_df = pd.read_sql_query(start, conn)
start_df['Year'] = start_df['Year'] = starting_year
end_df = pd.read_sql_query(end, conn)
end_df['Year'] = end_df['Year'] = ending_year

all_df = pd.concat([start_df, end_df])
new_df = pd.melt(all_df, id_vars=['Year'], value_vars=['Economy', 'Health', 'Freedom', 'Government'], var_name = "Category")
new_df = new_df.rename(columns = {'value': 'Category Score'})

category = pd.DataFrame({'Category': ['Economy', 'Health', 'Freedom', 'Government']})
selection = alt.selection_multi(fields=['Category'])
color = alt.condition(selection, alt.Color('Category:N'), alt.value('lightgray'))
make_selector = alt.Chart(category).mark_rect().encode(y='Category', color=color).add_selection(selection)
score_chart = alt.Chart(new_df).mark_line().encode(x='Year:N', y=alt.Y('Category Score:Q', scale=alt.Scale(domain=[0, 2])), color='Category').transform_filter(selection).properties(title = "Comparing Scores Affecting Happiness", width = 250)

make_selector | score_chart


alt.HConcatChart(...)

In [ ]:
# choose area instead of player
# country still showing switzerland
# change y-axis instead of value 
# add title